## consolidate predictions detail and get global metrics

In [1]:
import os
import json
import numpy as np
import pandas as pd
import joblib
from datetime import datetime
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [42]:
from bokeh.plotting import figure, show, output_file, save
from bokeh.io import output_notebook
from bokeh.layouts import row, gridplot, layout
from bokeh.palettes import d3
output_notebook()

Loading BokehJS ...

In [2]:
# symmetric mean absolute percentage error
def symmetric_mean_absolute_percentage_error(targets, predictions):
    '''
    predictions: a list with the predicted values
    targets: a list with the actual values
    '''
    import numpy as np
    # lists to NumPy arrays
    targets, predictions = np.array(targets), np.array(predictions)
    # verify predictions and targets have the same shape
    if predictions.shape == targets.shape:
            return(np.sum(2*np.abs(predictions - targets) /
                          (np.abs(targets) + np.abs(predictions)))/predictions.shape[0])

In [3]:
PROJECT_ROOT = '/home/developer/gcp/cbidmltsf'

In [63]:
model_ids = ['BSCTRFM_TPU_015',
             'BSCTRFM_TPU_016',
             'BSCTRFM_TPU_017',
             'BSCTRFM_TPU_018',
             'BSCTRFM_TPU_019',
             'BSCTRFM_TPU_020',
             'BSCTRFM_TPU_021']

In [67]:
# build a list with customer_ids
start, end = 320, 330

customer_ids = ['MT_{:03d}'.format(token_id) for token_id in np.arange(start, end + 1)]

In [70]:
# starting timestamps to predict over a complete day
start_row_ids = [0, 24, 48, 72, 96, 120, 144]

In [73]:
# create a dictionary to load predictions detail into, use saved models as primary keys
predictions_detail = dict()

summary = dict()

In [74]:
metrics = dict()

In [75]:
for model_id in model_ids[-1:]:
    
    metrics[model_id] = dict()
    
    # get the SLDB parameters for the forecasting model
    config_json_file = '{}/{}/{}.json'.format(PROJECT_ROOT,
                                              'parameters',
                                              model_id)

    # recover the sldb dictionary from the json file in parameters/
    with open(config_json_file, 'r') as inputfile:
        configuration = json.load(inputfile)
        
    sldb_id = configuration['data_dir'][21:]
    
    forecast_window = 24
    inference = '{:03d}'.format(forecast_window)
    
    # build a path to the SLDB json file
    data_dir = '{}/{}/{}'.format(PROJECT_ROOT, 'sldbs', sldb_id)
    
    # then get the ts_identifier from the json file in the sldb directory
    sldb_json_file = '{}/sldb.json'.format(data_dir)
    
    # open the json file
    with open(sldb_json_file, 'r') as inputfile:
        sldb_dict = json.load(inputfile)
        
    ts_identifier = sldb_dict['ts']
    m = sldb_dict['embedding']['hourly']
    t = sldb_dict['no_targets']

    execution = 0
    event = 0
    
    # get all available saved models from the model_id, execution combinations
    saved_models = os.listdir('{}/models/{}_{:02d}/export/exporter'.format(PROJECT_ROOT,
                                                                           model_id,
                                                                           execution))

    # sort the list
    saved_models.sort()
    
        
    # iterate on saved_models and customer_ids to load all predictions detail dataframes
    for saved_model in saved_models:
        
        predictions_detail[saved_model] = dict()
        
        for customer_id in customer_ids:
            detail_pickle_path = '{}/{}/{}/{}_{:02d}_{}_{}_{}_{:02d}.pkl'.format(
                PROJECT_ROOT,
                'database',
                'predictions_detail',
                model_id,
                execution,
                saved_model,
                # for electricity dataset, replace dataset with customer_id
                customer_id,
                inference,
                event)

            predictions_detail[saved_model][customer_id] = pd.read_pickle(detail_pickle_path)
            print('Loaded predictions from {} saved model over {} event {:02d}'.format(saved_model,
                                                                                   customer_id,
                                                                                   event))
        
    for saved_model in saved_models:

        summary[saved_model] = pd.DataFrame(
            columns=['saved_model', 'customer_id', 'timestamp', 'prediction', 'target'])

        for customer_id in customer_ids:

            # for each customer, iterate on starting rows for each day
            for start_row_id in start_row_ids:
                # make a buffer dataframe for one-day predictions
                buffer_df = pd.DataFrame()

                # populate the buffer dataframe

                # build a 24-time repeated list for the saved_model column
                buffer_df['saved_model'] = 24*[saved_model]

                # build a 24-time repeated list for the customer_id column
                buffer_df['customer_id'] = 24*[customer_id]
                # remember that predictions_detail dataframe stores lists,
                # then the index required to retrieve the list contents is the row index (find out why, later...)
                buffer_df['timestamp'] = pd.to_datetime(
                    predictions_detail[saved_model][customer_id]['string_timestamps'][start_row_id:start_row_id+1][start_row_id]
                )
                buffer_df['prediction'] = \
                    predictions_detail[saved_model][customer_id]['predictions'][start_row_id:start_row_id+1][start_row_id]

                buffer_df['target'] = \
                    predictions_detail[saved_model][customer_id]['targets'][start_row_id:start_row_id+1][start_row_id]

                # buffer_df = buffer_df.set_index('timestamp')

                summary[saved_model] = pd.concat([summary[saved_model], buffer_df])


                
    for saved_model in saved_models:
        
        metrics[model_id][saved_model]['mae'] = mean_absolute_error(
            summary[saved_model]['prediction'], summary[saved_model]['target'])

        metrics[model_id][saved_model]['true_values_average'] = np.mean(
            summary[saved_model]['target'])

        metrics[model_id][saved_model]['nd'] = metrics[model_id][saved_model]['mae']/metrics[model_id][saved_model]['true_values_average']

        metrics[model_id][saved_model]['rmse'] = sqrt(mean_squared_error(summary[saved_model]['prediction'],
                                                               summary[saved_model]['target']))

        metrics[model_id][saved_model]['nrmse'] = metrics[model_id][saved_model]['rmse']/metrics[model_id][saved_model]['true_values_average']

        metrics[model_id][saved_model]['smape'] = symmetric_mean_absolute_percentage_error(
            summary[saved_model]['prediction'], summary[saved_model]['target'])
                


Loaded predictions from 1633995397 saved model over MT_320 event 00
Loaded predictions from 1633995397 saved model over MT_321 event 00
Loaded predictions from 1633995397 saved model over MT_322 event 00
Loaded predictions from 1633995397 saved model over MT_323 event 00
Loaded predictions from 1633995397 saved model over MT_324 event 00
Loaded predictions from 1633995397 saved model over MT_325 event 00
Loaded predictions from 1633995397 saved model over MT_326 event 00
Loaded predictions from 1633995397 saved model over MT_327 event 00
Loaded predictions from 1633995397 saved model over MT_328 event 00
Loaded predictions from 1633995397 saved model over MT_329 event 00
Loaded predictions from 1633995397 saved model over MT_330 event 00
Loaded predictions from 1633995589 saved model over MT_320 event 00
Loaded predictions from 1633995589 saved model over MT_321 event 00
Loaded predictions from 1633995589 saved model over MT_322 event 00
Loaded predictions from 1633995589 saved model o

KeyError: '1633995397'

### over the 7 days of the test dataset, daily forecasting windows, starting at midnight (2014-09-0X 00:00:00)

In [43]:
plots = dict()

In [44]:
saved_models

['1633995397', '1633995589', '1633995770', '1633995965', '1633996143']

In [49]:
epochs = [10, 20, 30, 40, 50]

In [61]:
plots[model_id] = figure(
    title=model_id,
    plot_width=960,
    plot_height=320)

plots[model_id].grid.grid_line_alpha=0.3
plots[model_id].xaxis.axis_label = 'Epochs'
plots[model_id].yaxis.axis_label = 'Metric value'

plots[model_id].line(
    x=epochs,
    y=[metrics[saved_model]['nd'] for saved_model in saved_models],
    legend_label='ND',
    color='red'
)

plots[model_id].line(
    x=epochs,
    y=[metrics[saved_model]['nrmse'] for saved_model in saved_models],
    legend_label='NRMSE',
    color='green'
)

plots[model_id].line(
    x=epochs,
    y=[metrics[saved_model]['smape'] for saved_model in saved_models],
    legend_label='SMAPE',
    color='blue'
)

show(plots[model_id])

In [27]:
for saved_model in saved_models:
    mae = mean_absolute_error(summary[saved_model]['prediction'], summary[saved_model]['target'])
    true_values_average = np.mean(summary[saved_model]['target'])
    nd = mae/true_values_average
    rmse = sqrt(mean_squared_error(summary[saved_model]['prediction'], summary[saved_model]['target']))
    nrmse = rmse/true_values_average
    # ToDo: adjust the function to naming convention in other metric functions
    smape = symmetric_mean_absolute_percentage_error(
        summary[saved_model]['prediction'], summary[saved_model]['target'])
    
    print('For {}: ND {} NRMSE {} SMAPE {}'.format(saved_model, nd, nrmse, smape))
    
# BSCTRFM_TPU_021_00
# pos encoding 7-D (B) (age, hour-of-day, day-of-week)
# d_model = 256
# scaler MinMax
# base_learning_rate = 0.00250
# 10, 20, 30, 40, 50 epochs

For 1633995397: ND 0.07484296127067461 NRMSE 0.1583037843628051 SMAPE 0.06507185802244647
For 1633995589: ND 0.06471731266358517 NRMSE 0.1363525991052468 SMAPE 0.06083652014707437
For 1633995770: ND 0.07326104517228291 NRMSE 0.15251369089598868 SMAPE 0.0675747892241711
For 1633995965: ND 0.07488216190979485 NRMSE 0.15431331374334814 SMAPE 0.07127139097970446
For 1633996143: ND 0.07729011302015971 NRMSE 0.1583506110288202 SMAPE 0.0719994150362771


In [35]:
for saved_model in saved_models:
    mae = mean_absolute_error(summary[saved_model]['prediction'], summary[saved_model]['target'])
    true_values_average = np.mean(summary[saved_model]['target'])
    nd = mae/true_values_average
    rmse = sqrt(mean_squared_error(summary[saved_model]['prediction'], summary[saved_model]['target']))
    nrmse = rmse/true_values_average
    # ToDo: adjust the function to naming convention in other metric functions
    smape = symmetric_mean_absolute_percentage_error(
        summary[saved_model]['prediction'], summary[saved_model]['target'])
    
    print('For {}: ND {} NRMSE {} SMAPE {}'.format(saved_model, nd, nrmse, smape))
    
# BSCTRFM_TPU_020_00
# pos encoding 7-D (A) (hour-of-day, day-of-week, day-of-year)
# d_model = 256
# scaler MinMax
# base_learning_rate = 0.00250
# 10, 20, 30, 40, 50 epochs

For 1633965741: ND 0.06156267478797082 NRMSE 0.1289781077508348 SMAPE 0.06422151281136641
For 1633965933: ND 0.07697785753365227 NRMSE 0.15092006319443982 SMAPE 0.07943247518617652
For 1633966116: ND 0.0877389815836313 NRMSE 0.17053787392683492 SMAPE 0.0883111652444393
For 1633966307: ND 0.08962859261778565 NRMSE 0.17454572322578543 SMAPE 0.09124919544884065
For 1633966487: ND 0.09512947614911729 NRMSE 0.18250538380550954 SMAPE 0.09408561453714227


In [26]:
for saved_model in saved_models:
    mae = mean_absolute_error(summary[saved_model]['prediction'], summary[saved_model]['target'])
    true_values_average = np.mean(summary[saved_model]['target'])
    nd = mae/true_values_average
    rmse = sqrt(mean_squared_error(summary[saved_model]['prediction'], summary[saved_model]['target']))
    nrmse = rmse/true_values_average
    # ToDo: adjust the function to naming convention in other metric functions
    smape = symmetric_mean_absolute_percentage_error(
        summary[saved_model]['prediction'], summary[saved_model]['target'])
    
    print('For {}: ND {} NRMSE {} SMAPE {}'.format(saved_model, nd, nrmse, smape))
    
# BSCTRFM_TPU_019_00
# d_model = 256
# pos encoding 11-D (age, hour-of-day, day-of-week, day-of-month, day-of-year)
# scaler Standard
# base_learning_rate = 0.00250
# 20, 40, 60, 80, 100 epochs

For 1633866439: ND 0.15985262313005602 NRMSE 0.3000182473019328 SMAPE 0.18693156145690695
For 1633866672: ND 0.1587508481301783 NRMSE 0.29904159843923855 SMAPE 0.18639390111785156
For 1633866885: ND 0.15925643548480703 NRMSE 0.3003315165142588 SMAPE 0.18650515084476996
For 1633867090: ND 0.1594465960781185 NRMSE 0.3000147902470114 SMAPE 0.18663029155234018
For 1633867289: ND 0.1591792981771819 NRMSE 0.299726724167925 SMAPE 0.18656579880701007


In [28]:
for saved_model in saved_models:
    mae = mean_absolute_error(summary[saved_model]['prediction'], summary[saved_model]['target'])
    true_values_average = np.mean(summary[saved_model]['target'])
    nd = mae/true_values_average
    rmse = sqrt(mean_squared_error(summary[saved_model]['prediction'], summary[saved_model]['target']))
    nrmse = rmse/true_values_average
    # ToDo: adjust the function to naming convention in other metric functions
    smape = symmetric_mean_absolute_percentage_error(
        summary[saved_model]['prediction'], summary[saved_model]['target'])
    
    print('For {}: ND {} NRMSE {} SMAPE {}'.format(saved_model, nd, nrmse, smape))
    
# BSCTRFM_TPU_018_01
# pos encoding 11-D (age, hour-of-day, day-of-week, day-of-month, day-of-year)
# d_model = 256
# scaler MinMax
# base_learning_rate = 0.00250
# 20, 40, 60, 80, 100 epochs

For 1633702824: ND 0.0955685046522273 NRMSE 0.2124706560386861 SMAPE 0.08539946691424563
For 1633703035: ND 0.09666513709062684 NRMSE 0.21353028669336147 SMAPE 0.08713840624908574
For 1633703239: ND 0.09251758831339751 NRMSE 0.1998163642227151 SMAPE 0.08430812133642535
For 1633703442: ND 0.09358557458156151 NRMSE 0.20535866916626308 SMAPE 0.08440183695615831
For 1633703649: ND 0.09164572499476527 NRMSE 0.1955614999164243 SMAPE 0.08218987608043318


In [27]:
for saved_model in saved_models:
    mae = mean_absolute_error(summary[saved_model]['prediction'], summary[saved_model]['target'])
    true_values_average = np.mean(summary[saved_model]['target'])
    nd = mae/true_values_average
    rmse = sqrt(mean_squared_error(summary[saved_model]['prediction'], summary[saved_model]['target']))
    nrmse = rmse/true_values_average
    # ToDo: adjust the function to naming convention in other metric functions
    smape = symmetric_mean_absolute_percentage_error(
        summary[saved_model]['prediction'], summary[saved_model]['target'])
    
    print('For {}: ND {} NRMSE {} SMAPE {}'.format(saved_model, nd, nrmse, smape))
    
# BSCTRFM_TPU_018_00
# pos encoding 11-D (age, hour-of-day, day-of-week, day-of-month, day-of-year)
# d_model = 256
# scaler MinMax
# base_learning_rate = 0.00250
# 20, 40, 60, 80, 100 epochs

For 1633701798: ND 0.0941519998088359 NRMSE 0.19424650703835225 SMAPE 0.08430341009566043
For 1633702002: ND 0.09750998147935375 NRMSE 0.1959272821139588 SMAPE 0.08764841440694712
For 1633702207: ND 0.09548022028712001 NRMSE 0.19556656884903484 SMAPE 0.08468963750948486
For 1633702423: ND 0.09896150024850345 NRMSE 0.1985769558610708 SMAPE 0.08789245902112117
For 1633702633: ND 0.09652474413864003 NRMSE 0.19616918847708367 SMAPE 0.08530757772254514


In [27]:
for saved_model in saved_models:
    mae = mean_absolute_error(summary[saved_model]['prediction'], summary[saved_model]['target'])
    true_values_average = np.mean(summary[saved_model]['target'])
    nd = mae/true_values_average
    rmse = sqrt(mean_squared_error(summary[saved_model]['prediction'], summary[saved_model]['target']))
    nrmse = rmse/true_values_average
    # ToDo: adjust the function to naming convention in other metric functions
    smape = symmetric_mean_absolute_percentage_error(
        summary[saved_model]['prediction'], summary[saved_model]['target'])
    
    print('For {}: ND {} NRMSE {} SMAPE {}'.format(saved_model, nd, nrmse, smape))
    
# BSCTRFM_TPU_017_00
# pos encoding 9-D (hour-of-day, day-of-week, day-of-month, day-of-year)
# d_model = 512
# scaler MinMax
# base_learning_rate = 0.00125
# 20, 40, 60, 80, 100, 120, 140, 160, 200 epochs

For 1633532109: ND 0.08391968908023607 NRMSE 0.17308211279928934 SMAPE 0.08098911367533751
For 1633532387: ND 0.08247158391330808 NRMSE 0.17175445896947505 SMAPE 0.07789510358854744
For 1633532611: ND 0.08373632470080083 NRMSE 0.1757226438318765 SMAPE 0.07659722293803967
For 1633532845: ND 0.08943910885790138 NRMSE 0.18770457151282385 SMAPE 0.07845570142924825
For 1633533083: ND 0.08563218078525504 NRMSE 0.1809419067948103 SMAPE 0.07538160878360324


In [27]:
for saved_model in saved_models:
    mae = mean_absolute_error(summary[saved_model]['prediction'], summary[saved_model]['target'])
    true_values_average = np.mean(summary[saved_model]['target'])
    nd = mae/true_values_average
    rmse = sqrt(mean_squared_error(summary[saved_model]['prediction'], summary[saved_model]['target']))
    nrmse = rmse/true_values_average
    # ToDo: adjust the function to naming convention in other metric functions
    smape = symmetric_mean_absolute_percentage_error(
        summary[saved_model]['prediction'], summary[saved_model]['target'])
    
    print('For {}: ND {} NRMSE {} SMAPE {}'.format(saved_model, nd, nrmse, smape))

# BSCTRFM_TPU_016_00
# pos encoding 9-D (hour-of-day, day-of-week, day-of-month, day-of-year)
# d_model = 512
# scaler MinMax
# base_learning_rate = 0.00250
# 20, 40, 60, 80, 100, 120, 140, 160, 200 epochs

For 1633530953: ND 0.08341669712207045 NRMSE 0.16672929151702992 SMAPE 0.08262868805490758
For 1633531183: ND 0.07944599314612216 NRMSE 0.16264175093007396 SMAPE 0.07736881092321744
For 1633531426: ND 0.07898689855531715 NRMSE 0.16146557623058522 SMAPE 0.0781048874007399
For 1633531652: ND 0.08532505922027828 NRMSE 0.1738145026053602 SMAPE 0.07989042103000737
For 1633531877: ND 0.08225182547980808 NRMSE 0.16825302801119876 SMAPE 0.07867756105630087


In [27]:
for saved_model in saved_models:
    mae = mean_absolute_error(summary[saved_model]['prediction'], summary[saved_model]['target'])
    true_values_average = np.mean(summary[saved_model]['target'])
    nd = mae/true_values_average
    rmse = sqrt(mean_squared_error(summary[saved_model]['prediction'], summary[saved_model]['target']))
    nrmse = rmse/true_values_average
    # ToDo: adjust the function to naming convention in other metric functions
    smape = symmetric_mean_absolute_percentage_error(
        summary[saved_model]['prediction'], summary[saved_model]['target'])
    
    print('For {}: ND {} NRMSE {} SMAPE {}'.format(saved_model, nd, nrmse, smape))

# BSCTRFM_TPU_015_02
# pos encoding 9-D (hour-of-day, day-of-week, day-of-month, day-of-year)
# d_model = 256
# scaler MinMax
# base_learning_rate = 0.00250
# 20, 40, 60, 80, 100, 120, 140, 160, 200 epochs

For 1633474201: ND 0.07375831660536315 NRMSE 0.15230470970086646 SMAPE 0.07137588257117915
For 1633474404: ND 0.07895120337066364 NRMSE 0.16923699614029766 SMAPE 0.0714609550542299
For 1633474605: ND 0.08218558863158312 NRMSE 0.17675654988116618 SMAPE 0.07256537008866383
For 1633474816: ND 0.08300970968684897 NRMSE 0.17948334228814225 SMAPE 0.07250197376985855
For 1633475020: ND 0.08001419853662056 NRMSE 0.17538226051637926 SMAPE 0.07126670351220397
For 1633475219: ND 0.07982681697878112 NRMSE 0.17475582968455947 SMAPE 0.07032845461160454
For 1633475426: ND 0.08008514367999972 NRMSE 0.17473123265209445 SMAPE 0.06960701521577986
For 1633475628: ND 0.08292089939378991 NRMSE 0.1840270460315019 SMAPE 0.07026546935420891
For 1633475832: ND 0.08265426789303487 NRMSE 0.17979784469048696 SMAPE 0.07169182791765653
For 1633476036: ND 0.08401720637508019 NRMSE 0.18601580742665286 SMAPE 0.0706546648203347


In [27]:
for saved_model in saved_models:
    mae = mean_absolute_error(summary[saved_model]['prediction'], summary[saved_model]['target'])
    true_values_average = np.mean(summary[saved_model]['target'])
    nd = mae/true_values_average
    rmse = sqrt(mean_squared_error(summary[saved_model]['prediction'], summary[saved_model]['target']))
    nrmse = rmse/true_values_average
    # ToDo: adjust the function to naming convention in other metric functions
    smape = symmetric_mean_absolute_percentage_error(
        summary[saved_model]['prediction'], summary[saved_model]['target'])
    
    print('For {}: ND {} NRMSE {} SMAPE {}'.format(saved_model, nd, nrmse, smape))

# BSCTRFM_TPU_015_00
# pos encoding 9-D (hour-of-day, day-of-week, day-of-month, day-of-year)
# d_model = 256
# scaler MinMax
# base_learning_rate = 0.00250
# 20, 40, 60, 80, 100, 120, 140, 160, 200 epochs

For 1633196871: ND 0.08412829704166151 NRMSE 0.17959798252230771 SMAPE 0.08071277302305699
For 1633197126: ND 0.08178093963423284 NRMSE 0.17079970153772212 SMAPE 0.07781654540046554
For 1633197345: ND 0.08265768871755412 NRMSE 0.18418564602796936 SMAPE 0.0764088103296674
For 1633197575: ND 0.07746174995853501 NRMSE 0.16554976676107486 SMAPE 0.0746029714810412
For 1633197797: ND 0.07678363903143116 NRMSE 0.15833885384093913 SMAPE 0.07427194099791928
For 1633198016: ND 0.08012344564720517 NRMSE 0.1678176502012755 SMAPE 0.07584769045565051
For 1633198227: ND 0.08236143555660547 NRMSE 0.1720318129905628 SMAPE 0.07529189986950587
For 1633198439: ND 0.07895855667595755 NRMSE 0.1619844851494527 SMAPE 0.07387324516995354
For 1633198645: ND 0.08111112175667977 NRMSE 0.1665733658166654 SMAPE 0.07406590245136283
For 1633198846: ND 0.07947216189815492 NRMSE 0.16195942604251842 SMAPE 0.07333527973799918
